In [1]:
import os
import numpy as np
import pandas as pd
import tensorflow as tf
from glob import glob
import cv2
from sklearn.model_selection import train_test_split


In [2]:

def load_data(path,split=0.2):
    images=sorted(glob(os.path.join(path,'Original/*.tif')))
    labels=sorted(glob(os.path.join(path,'GroundTruth/*.tif')))
    total_size=len(images)
    valid_size=int(split*total_size)
    test_size=int(split*total_size)

    train_x,valid_x=train_test_split(images,test_size=valid_size,random_state=42)
    train_y,valid_y=train_test_split(labels,test_size=valid_size,random_state=42)

    train_x,test_x=train_test_split(train_x,test_size=test_size,random_state=42)
    train_y,test_y=train_test_split(train_y,test_size=test_size,random_state=42)

    return (train_x,train_y),(valid_x,valid_y),(test_x,test_y)

In [3]:
def read_image(path):
    path=path.decode()
    x=cv2.imread(path,cv2.IMREAD_COLOR)
    x=cv2.resize(x,(256,256))
    x=x/255.0
    return x

def read_mask(path):
    path=path.decode()
    x=cv2.imread(path,cv2.IMREAD_GRAYSCALE)
    x=cv2.resize(x,(256,256))
    x=x/255.0
    x=np.expand_dims(x,axis=-1)
    return x

In [4]:
def mask_parse(mask):
    mask=np.squeeze(mask)
    mask=[mask,mask,mask]
    mask=np.transpose(mask,(1,2,0))
    return mask

In [5]:
def tf_parse(x,y):
    def _parse(x,y):
        x=read_image(x)
        y=read_mask(y)
        return x,y
    x,y=tf.numpy_function(_parse,[x,y],[tf.float64,tf.float64])
    x.set_shape([256,256,3])
    y.set_shape([256,256,1])
    return x,y

In [6]:
def dataset(x,y,batch=8):
    dataset=tf.data.Dataset.from_tensor_slices((x,y))
    dataset=dataset.map(tf_parse)
    dataset=dataset.batch(batch)
    dataset=dataset.repeat()
    dataset=dataset.prefetch(2)
    return dataset 

In [7]:
path="CVC-ClinicDB"
(train_x,train_y),(valid_x,valid_y),(test_x,test_y)=load_data(path)

print(f"Train: {len(train_x)} - {len(train_y)}")
print(f"Valid: {len(valid_x)} - {len(valid_y)}")
print(f"Test: {len(test_x)} - {len(test_y)}")


Train: 368 - 368
Valid: 122 - 122
Test: 122 - 122


In [8]:
ds=dataset(test_x,test_y,8)
for x,y in ds:
    print(x.shape,y.shape)
    break

(8, 256, 256, 3) (8, 256, 256, 1)


In [9]:
from tensorflow.keras.layers import Conv2D,Input,BatchNormalization,Activation,Dropout,MaxPooling2D,Conv2DTranspose,UpSampling2D,concatenate,Flatten
from tensorflow.keras.models import Model

In [10]:
def conv_block(x,num_filters):
    x=Conv2D(num_filters,3,padding="same")(x)
    x=BatchNormalization()(x)
    x=Activation("relu")(x)

    x=Conv2D(num_filters,3,padding="same")(x)
    x=BatchNormalization()(x)
    x=Activation("relu")(x)

    return x

In [11]:
def build_mode():
    size=256
    num_filters=[16,32,48,64]
    inputs=Input((size,size,3))

    skip_x=[]

    x=inputs
    for f in num_filters:
        x=conv_block(x,f)
        skip_x.append(x)
        x=MaxPooling2D((2,2))(x)

    x=conv_block(x,num_filters[-1])

    num_filters.reverse()
    skip_x.reverse()

    for i,f in enumerate(num_filters):
        x=UpSampling2D((2,2))(x)
        xs=skip_x[i]
        x=concatenate([x,xs])
        x=conv_block(x,f)
        

    x=Conv2D(1,(1,1),padding="same")(x)
    x=Activation("sigmoid")(x)

    return Model(inputs,x) 

In [12]:
model=build_mode()
model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_1 (InputLayer)        [(None, 256, 256, 3)]        0         []                            
                                                                                                  
 conv2d (Conv2D)             (None, 256, 256, 16)         448       ['input_1[0][0]']             
                                                                                                  
 batch_normalization (Batch  (None, 256, 256, 16)         64        ['conv2d[0][0]']              
 Normalization)                                                                                   
                                                                                                  
 activation (Activation)     (None, 256, 256, 16)         0         ['batch_normalization[0][0

In [13]:
os.environ["TF_CPP_MIN_LOG_LEVEL"]="2"
from tensorflow.keras.losses import binary_crossentropy
from tensorflow.keras.callbacks import ModelCheckpoint,CSVLogger,ReduceLROnPlateau,EarlyStopping,TensorBoard
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.metrics import Recall,Precision

In [14]:
def iou(y_true,y_pred):
    def f(y_true,y_pred):
        intersection=(y_true*y_pred).sum()
        union=y_true.sum()+y_pred.sum()-intersection
        x=(intersection+1e-15)/(union+1e-15)
        x=x.astype(np.float32)
        return x
    return tf.numpy_function(f,[y_true,y_pred],tf.float32)

In [15]:
# hyperparameters
lr=1e-4
batch_size=8
epochs=20

train_dataset=dataset(train_x,train_y,batch=batch_size)
valid_dataset=dataset(valid_x,valid_y,batch=batch_size)

opt=tf.keras.optimizers.Adam(lr)
metrics=["acc",tf.keras.metrics.Recall(),tf.keras.metrics.Precision(),iou]
model.compile(loss=binary_crossentropy,optimizer=opt,metrics=metrics)

callbacks=[
    ModelCheckpoint("files/model.h5"),
    ReduceLROnPlateau(monitor='val_loss',factor=0.1,patience=3),
    CSVLogger("files/data.csv"),
    TensorBoard(),
    EarlyStopping(monitor='val_loss',patience=10,restore_best_weights=False)
]

train_steps=len(train_x)//batch_size
valid_steps=len(valid_x)//batch_size

if len(train_x)%batch_size!=0:
    train_steps+=1
if len(valid_x)%batch_size!=0:
    valid_steps+=1

model.fit(
    train_dataset,
    validation_data=valid_dataset,
    epochs=epochs,
    steps_per_epoch=train_steps,
    validation_steps=valid_steps,
    callbacks=callbacks
)


Epoch 1/20
46/46 [==============================] - ETA: 0s - loss: 0.5988 - acc: 0.7792 - recall: 0.3708 - precision: 0.1880 - iou: 0.0916

c:\Users\KIIT\anaconda3\Lib\site-packages\keras\src\engine\training.py:3000: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


46/46 [==============================] - 225s 5s/step - loss: 0.5988 - acc: 0.7792 - recall: 0.3708 - precision: 0.1880 - iou: 0.0916 - val_loss: 0.6756 - val_acc: 0.6976 - val_recall: 0.0063 - val_precision: 0.0025 - val_iou: 0.0722 - lr: 1.0000e-04
Epoch 2/20
46/46 [==============================] - 220s 5s/step - loss: 0.4905 - acc: 0.8858 - recall: 0.3228 - precision: 0.4040 - iou: 0.0990 - val_loss: 0.6241 - val_acc: 0.8993 - val_recall: 1.2272e-04 - val_precision: 6.9380e-04 - val_iou: 0.0684 - lr: 1.0000e-04
Epoch 3/20
46/46 [==============================] - 246s 5s/step - loss: 0.4411 - acc: 0.8999 - recall: 0.3961 - precision: 0.4941 - iou: 0.1116 - val_loss: 0.5679 - val_acc: 0.9108 - val_recall: 4.3829e-05 - val_precision: 0.0010 - val_iou: 0.0684 - lr: 1.0000e-04
Epoch 4/20
46/46 [==============================] - 258s 6s/step - loss: 0.4061 - acc: 0.9128 - recall: 0.4762 - precision: 0.5745 - iou: 0.1281 - val_loss: 0.5109 - val_acc: 0.9140 - val_recall: 2.3375e-05 - val_

In [16]:

model.save("files/model.h5")


In [21]:
from tensorflow.keras.utils import CustomObjectScope
from tqdm import tqdm

In [19]:
test_dataset=dataset(test_x,test_y,batch=8)
test_steps=len(test_x)//8
if len(test_x)%8!=0:
    test_steps+=1

with CustomObjectScope({'iou':iou}):
    model=tf.keras.models.load_model("files/model.h5")

model.evaluate(test_dataset,steps=test_steps)




16/16 [==============================] - 16s 941ms/step - loss: 0.2804 - acc: 0.9319 - recall: 0.4953 - precision: 0.7119 - iou: 0.1817


[0.28040361404418945,
 0.931876540184021,
 0.4953034222126007,
 0.7119431495666504,
 0.18167677521705627]

In [20]:
def read_image(path):
    x=cv2.imread(path,cv2.IMREAD_COLOR)
    x=cv2.resize(x,(256,256))
    x=x/255.0
    return x

def read_mask(path):
    x=cv2.imread(path,cv2.IMREAD_GRAYSCALE)
    x=cv2.resize(x,(256,256))
    x=x/255.0
    x=np.expand_dims(x,axis=-1)
    return x

def mask_parse(mask):
    mask=np.squeeze(mask)
    mask=[mask,mask,mask]
    mask=np.transpose(mask,(1,2,0))
    return mask

In [25]:
for i,(x,y) in tqdm(enumerate(zip(test_x,test_y)),total=len(test_x)):
    x=read_image(x)
    y=read_mask(y)
    y_pred=model.predict(np.expand_dims(x,axis=0))[0]>0.5
    h,w,_=x.shape
    white_line=np.ones((h,10,3))*255.0

    all_images= [
        x*255.0,white_line,mask_parse(y),white_line,mask_parse(y_pred)
    ]
    image= np.concatenate(all_images,axis=1)
    cv2.imwrite(f"files/test/{i}.png",image*255.0)


  0%|          | 0/122 [00:00<?, ?it/s]

1/1 [==============================] - 0s 155ms/step


  1%|          | 1/122 [00:00<00:30,  3.92it/s]

1/1 [==============================] - 0s 116ms/step


  2%|▏         | 2/122 [00:00<00:27,  4.39it/s]

...

 98%|█████████▊| 120/122 [00:27<00:00,  4.05it/s]

1/1 [==============================] - 0s 194ms/step


 99%|█████████▉| 121/122 [00:27<00:00,  3.83it/s]

1/1 [==============================] - 0s 154ms/step


100%|██████████| 122/122 [00:27<00:00,  4.42it/s]
